In [0]:
def drop_rescued_data_columns(spark, catalog):
    spark.sql(f'use catalog {catalog}')
    system_schemas = ['information_schema', 'sys']
    schemas = [row.databaseName for row in spark.sql(f'SHOW SCHEMAS IN {catalog}').collect()]
    for schema in schemas:
        if schema in system_schemas:
            print(f'Skipping system schema {schema}')
            continue
        tables = spark.sql(f'SHOW TABLES IN {catalog}.{schema}').filter('isTemporary = false')
        view_rows = spark.sql(f'SHOW VIEWS IN {catalog}.{schema}').collect()
        views = {row.viewName for row in view_rows}
        for row in tables.collect():
            name = row.tableName
            if name in views:
                print(f'Skipping view {catalog}.{schema}.{name}')
                continue
            columns = spark.table(f'{catalog}.{schema}.{name}').columns
            if '_rescued_data' in columns:
                spark.sql(f'ALTER TABLE {catalog}.{schema}.{name} SET TBLPROPERTIES (delta.columnMapping.mode="name")')
                spark.sql(f'ALTER TABLE {catalog}.{schema}.{name} DROP COLUMN _rescued_data')
                print(f'Dropped _rescued_data from {catalog}.{schema}.{name}')
            else:
                print(f'Skipping {catalog}.{schema}.{name}: no _rescued_data')

drop_rescued_data_columns(spark, 'edsm')